Автор: Павел Любовин    
chistota@mail.ru   

Продолжение.   
Начало см. в ноутбуке: Cart_forecast.ipynb

# Прогноз с помощью LSTM

Основная идея:  
Последовательность заказов от одного пользователя - это временной ряд.  
Если превратить заказ в вектор, то ряд таких векторов можно попробовать предсказать  
с помощью реккурентных нейронных сетей.  
В данном решении была выбрана LSTM.

In [253]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn

In [254]:
df = pd.read_csv('train.csv')

In [255]:
df.head()

,user_id,order_completed_at,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198
3,2,2015-03-22 09:25:46,88
4,2,2015-03-22 09:25:46,157


In [256]:
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])  # Привожу к дате-времени

In [257]:
sample_out_df = pd.read_csv('sample_submission.csv')

In [258]:
sample_out_df.head()

,id,target
0,0;133,0
1,0;5,1
2,0;10,0
3,0;396,1
4,0;14,0


# Подготовка данных

In [259]:
# Выделяю пользователя и товар в отдельные колонки
sample_out_df['user_category'] = sample_out_df['id'].apply(lambda x: x.split(';'))
sample_out_df['user'] = sample_out_df['user_category'].apply(lambda x: int(x[0]))
sample_out_df['category'] = sample_out_df['user_category'].apply(lambda x: int(x[1]))

In [260]:
sample_out_df.head()

,id,target,user_category,user,category
0,0;133,0,"[0, 133]",0,133
1,0;5,1,"[0, 5]",0,5
2,0;10,0,"[0, 10]",0,10
3,0;396,1,"[0, 396]",0,396
4,0;14,0,"[0, 14]",0,14


In [261]:
out_categories = pd.unique(sample_out_df['category'])

In [262]:
out_categories = np.sort(out_categories).tolist()

In [263]:
print('Количество уникальных категорий:', len(out_categories))

Количество уникальных категорий: 858


In [264]:
# Удаляю из основного фрейма все категории, которые не требуется предсказывать
# т.е. тех, которых нет в out_categories
df_cut = df[df['cart'].isin(out_categories)]

In [265]:
# Формирую датасет с заказами:
orders_df = df_cut.groupby(['user_id', 'order_completed_at'])['cart'].apply(set).to_frame().reset_index()

In [266]:
orders_df.head()

,user_id,order_completed_at,cart
0,0,2020-07-19 09:59:17,"{14, 430, 82, 20, 405, 441, 379, 57}"
1,0,2020-08-24 08:55:32,"{5, 133, 10, 396, 14, 402, 405, 22, 409, 25, 2..."
2,0,2020-09-02 07:38:25,"{803, 169, 170, 398, 399, 401, 84, 55, 440, 57..."
3,1,2019-05-08 16:09:41,{55}
4,1,2020-01-17 14:44:23,"{421, 204, 82, 86, 55, 798}"


In [267]:
orders_df['index_cart'] = orders_df['cart'].apply(lambda x: [out_categories.index(i) for i in x])

In [268]:
#  Заготовка под вектор заказов (пока только нули)
orders_df['vector_cart'] = pd.Series([[0] * len(out_categories) for _ in range(len(orders_df))])

In [269]:
# Функция, расставляющая единицы по заданному списку индексов
def index_ones(index_cart, vector_cart):
    for i in index_cart:
        vector_cart[i] = 1
    return vector_cart

In [270]:
# Расстановка единиц с помощью спец. функции (чуть выше ее определение)
# В итоге из каждого заказа получился вектор размерности 858 (по количеству товарных категорий)
orders_df['vector_cart'] = orders_df.apply(lambda x: index_ones(x.index_cart, x.vector_cart), axis=1)

In [271]:
orders_df.head()

,user_id,order_completed_at,cart,index_cart,vector_cart
0,0,2020-07-19 09:59:17,"{14, 430, 82, 20, 405, 441, 379, 57}","[14, 425, 81, 20, 400, 436, 374, 56]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
1,0,2020-08-24 08:55:32,"{5, 133, 10, 396, 14, 402, 405, 22, 409, 25, 2...","[5, 132, 10, 391, 14, 397, 400, 22, 404, 25, 2...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, ..."
2,0,2020-09-02 07:38:25,"{803, 169, 170, 398, 399, 401, 84, 55, 440, 57...","[781, 168, 169, 393, 394, 396, 83, 54, 435, 56...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,2019-05-08 16:09:41,{55},[54],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1,2020-01-17 14:44:23,"{421, 204, 82, 86, 55, 798}","[416, 201, 81, 85, 54, 776]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Формирование обучающей выборки

Теперь у нас есть вектор для каждого заказа.  
Из этих векторов будем формировать обучающие примеры и таргет, который надо предсказать.  
Обучающими примерами будут 10 (например) подряд идущих заказов, а одиннадцатый заказ - это таргет.  
Если у какого-то пользователя более 11 заказов, то для него удастся сформировать несколько обучающих примеров
(кол-во заказов минус 11).  
Если кол-во заказов менее 11, то все равно сформируем последовательность из 11 заказов, добавив в начало
несколько пустых заказов (пустой заказ - это вектор, состоящий из одних нулей)  

In [610]:
# Кол-во заказов для для тренировочной последовательности
ORDERS_N = 10

In [611]:
# Уникальные пользователи в заказах
user_ids = orders_df['user_id'].unique()

In [612]:
# Нулевой (пустой) фрейм (1 строка)
empty_dict = {'user_id': 0, 'order_completed_at': pd.to_datetime('2000-01-01 00:00:00'),
                        'cart': {}, 'index_cart': [], 'vector_cart': [0] * len(out_categories)}
empty_df = pd.DataFrame({k: [v] for k, v in empty_dict.items()})

In [613]:
empty_df

,user_id,order_completed_at,cart,index_cart,vector_cart
0,0,2000-01-01,{},[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [614]:
# Функция выравнивания кол-ва последовательности заказов, если она меньше чем ORDERS_N + 1
def df_arrange(user_df, null_df):
    if (len(user_df) <= ORDERS_N):
        k = ORDERS_N + 1 - len(user_df)  # Сколько нулевых заказов надо вставить в начало фрейма
        for _ in range(k):
            user_df = pd.concat([null_df, user_df], ignore_index=True)  # Добавление нулевого заказа в начало
    return user_df

In [615]:
# Функция формирует список кортежей: (тренировочные примеры равной длины, таргет, дату последнего заказа)
def create_inout_sequences(input_df, tw):
    inout_seq = []
    L = len(input_df)
    input_df = input_df.to_numpy()
    for i in range(L-tw):
        train_seq = input_df[i:i+tw, 1].tolist()
        train_label = input_df[i+tw, 1]
        last_order_completed_at = input_df[i+tw, 0]
        inout_seq.append((train_seq ,train_label, last_order_completed_at))
    return inout_seq
# Тут я кроме фич и таргета добавил дату (3-й элемент в кортежах).

In [616]:
# Подготовка тренировочных примеров по всем пользователям
# Информация усредняется по всем пользователям
counter = 0

inout_seq = []
with tqdm(total=len(user_ids)) as pbar:
    for user_id in user_ids:

        counter += 1
        user_df = orders_df[orders_df['user_id'] == user_id]
        user_df.sort_values(by='order_completed_at')
        empty_df.loc[0, 'user_id'] = user_id
        user_df = df_arrange(user_df, empty_df).reset_index(drop=True)
        inout_seq.extend(create_inout_sequences(user_df.loc[:, ['order_completed_at', 'vector_cart']], ORDERS_N))

        # Пока для тестов остановимся на заказах только от первых 1000 пользователей (из 20тыс)
        # if counter == 1000:
        #    break
        

        pbar.update()

100%|██████████| 20000/20000 [01:28<00:00, 224.95it/s]


In [617]:
# Формирование тренировочной и тестовой выборки
L = len(inout_seq)
T = int(L * 0.8)
# Сортровка по 3-му столбцу (last_order_completed_at)
inout_seq.sort(key=lambda x:x[2])

# Разбиение на тренировочную и тестовую выборку не нужно
# train_X = torch.FloatTensor(np.array(inout_seq[:T])[:, 0].tolist())
# train_Y = torch.FloatTensor(np.array(inout_seq[:T])[:, 1].tolist())
# test_X = torch.FloatTensor(np.array(inout_seq[T:])[:, 0].tolist())
# test_Y = torch.FloatTensor(np.array(inout_seq[T:])[:, 1].tolist())

# Буду учиться на всех имеющихся данных
train_X = torch.FloatTensor(np.array(inout_seq)[:, 0].tolist())
train_Y = torch.FloatTensor(np.array(inout_seq)[:, 1].tolist())

# Модель

In [618]:
class LSTM_my(nn.Module):
    def __init__(self, input_size=len(out_categories), hidden_layer_size=1000, output_size=len(out_categories)):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        
#         prediction = self.sigmoid(predictions[-1])
        prediction = predictions[-1]
    
        return prediction

In [619]:
model_my = LSTM_my()
loss_function_my = nn.MSELoss()
# loss_function_my = nn.BCELoss()
# loss_function_my = nn.functional.binary_cross_entropy()
optimizer_my = torch.optim.Adam(model_my.parameters(), lr=0.001)

In [620]:
print(model_my)

LSTM_my(
  (lstm): LSTM(858, 1000)
  (linear): Linear(in_features=1000, out_features=858, bias=True)
  (sigmoid): Sigmoid()
)


In [621]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
device

'cpu'

In [622]:
model_my.to(device)
train_X = train_X.to(device)
train_Y = train_Y.to(device)

# Обучение модели

In [628]:
epochs = 1
train_part = 0.1  # Доля тренировочных данных, участвующих в обучении

for i in range(epochs):
#     for seq, labels in train_inout_seq_my:
    with tqdm(total=round(len(train_X) * train_part)) as pbar:
        for i in range(round(len(train_X) * train_part)):
            optimizer_my.zero_grad()
            model_my.hidden_cell = (torch.zeros(1, 1, model_my.hidden_layer_size).to(device),
                            torch.zeros(1, 1, model_my.hidden_layer_size).to(device))

#             y_pred = model_my(train_X[i])
            y_pred = model_my(train_X[len(train_X) - i - 1])  # Чтобы брать последние примеры (по дате)
            
            single_loss_my = loss_function_my(y_pred, train_Y[i])
#             single_loss_my = nn.functional.binary_cross_entropy(y_pred, train_Y[i])

            single_loss_my.backward()
            optimizer_my.step()
            
            pbar.update()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss_my.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss_my.item():10.10f}')

100%|██████████| 9092/9092 [20:42<00:00,  7.31it/s]

epoch: 9091 loss: 0.0076208692


In [629]:
# Сохранение параметров обученной модели
torch.save(model_my.state_dict(), 'LSTM_model_state_dict14.pt')

In [630]:
# Загрузка параметров модели
model_my.load_state_dict(torch.load('LSTM_model_state_dict14.pt'))
model_my.eval()

LSTM_my(
  (lstm): LSTM(858, 1000)
  (linear): Linear(in_features=1000, out_features=858, bias=True)
  (sigmoid): Sigmoid()
)

# Прогноз корзины для каждого пользователя

Нам нужны последние 10 заказов каждого пользователя  
Добавить в начало пустые заказы, если их менее 10  
Сделать прогноз по каждому пользователю  
Из прогноза сделать submission  

In [631]:
# Этот параметр уже задан ранее
# Кол-во заказов, по которому будем обучаться или предсказывать следующий заказ
# ORDERS_N = 10

In [632]:
model_my.eval()

LSTM_my(
  (lstm): LSTM(858, 1000)
  (linear): Linear(in_features=1000, out_features=858, bias=True)
  (sigmoid): Sigmoid()
)

In [633]:
# Порог для отнесения к 1-му классу
TRESHOLD = 0.20

predictions = {}
predictions_raw = {}
with tqdm(total=len(user_ids)) as pbar:
    for user_id in user_ids:
        # Подготовка последних заказов для каждого пользователя
        user_df = orders_df[orders_df['user_id'] == user_id]
        user_df.sort_values(by='order_completed_at')
        empty_df.loc[0, 'user_id'] = user_id
        user_df = df_arrange(user_df, empty_df).reset_index(drop=True)
        # Оставляю только последние заказы
        user_df = user_df.iloc[-ORDERS_N:]
        
        # Делаю прогноз
        with torch.no_grad():
            model_my.hidden = (torch.zeros(1, 1, model_my.hidden_layer_size).to(device),
                            torch.zeros(1, 1, model_my.hidden_layer_size).to(device))
            X = torch.FloatTensor(user_df.loc[:, ['vector_cart']].values.tolist()).to(device)
            # Предсказание в сыром виде - просто какие-то числа
            pred = model_my(X)
            # Для чисел выше заданного порога определяю категорию - это и есть прогноз модели
            pred_2 = [out_categories[i] for i, el in enumerate(pred) if el >= TRESHOLD]
            predictions[user_id] = pred_2
            predictions_raw[user_id] = pred
            
        pbar.update()

100%|██████████| 20000/20000 [11:52<00:00, 28.06it/s]  


In [653]:
# # Пробую потюнить порог
# TRESHOLD = 0.15
# for user_id in predictions_raw:
#     predictions[user_id] = [out_categories[i] for i, el in enumerate(predictions_raw[user_id]) if el >= TRESHOLD]

In [667]:
CUR_USER_ID = 5

In [668]:
# Пример прогноза для пользователя №3 (user_id == 3)
predictions[CUR_USER_ID]

[9,
 14,
 16,
 17,
 19,
 22,
 23,
 29,
 41,
 54,
 55,
 57,
 61,
 84,
 88,
 170,
 376,
 382,
 396,
 398,
 402,
 409,
 420,
 430]

In [669]:
# Кол-во заказов в корзине пользователя
len(predictions[CUR_USER_ID])

24

In [657]:
# Вот заказы этого пользователя:
orders_df[orders_df['user_id'] == CUR_USER_ID]

,user_id,order_completed_at,cart,index_cart,vector_cart
27,3,2015-06-18 16:15:33,{399},[394],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
28,3,2015-07-04 14:05:22,{399},[394],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
29,3,2015-08-12 10:33:44,"{804, 134}","[782, 133]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
30,3,2015-11-27 19:37:17,{399},[394],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
31,3,2020-04-12 10:57:57,"{398, 15, 16, 19, 22, 27, 412, 41, 43, 430, 17...","[393, 15, 16, 19, 22, 27, 407, 40, 42, 425, 17...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
32,3,2020-06-12 14:49:01,"{398, 15, 16, 402, 19, 404, 148, 411, 41, 42, ...","[393, 15, 16, 397, 19, 399, 147, 406, 40, 41, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
33,3,2020-06-24 13:07:29,"{64, 41, 43, 14, 92, 142, 17, 19, 179, 22, 57,...","[63, 40, 42, 14, 91, 141, 17, 19, 178, 22, 56,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


# Делаем submissions

In [658]:
sample_out_df = pd.read_csv('sample_submission.csv')
#Найдем всех уникальных пользователей и товары в выходном файле (т.е. те сущности, которые надо предсказать)
sample_out_df['user_category'] = sample_out_df['id'].apply(lambda x: x.split(';'))
sample_out_df['user'] = sample_out_df['user_category'].apply(lambda x: int(x[0]))
sample_out_df['category'] = sample_out_df['user_category'].apply(lambda x: int(x[1]))

In [659]:
sample_out_df.head()

,id,target,user_category,user,category
0,0;133,0,"[0, 133]",0,133
1,0;5,1,"[0, 5]",0,5
2,0;10,0,"[0, 10]",0,10
3,0;396,1,"[0, 396]",0,396
4,0;14,0,"[0, 14]",0,14


In [660]:
# Обновления таргетов в соответствиями с предсказаниями нейросети:
sample_out_df['target'] = sample_out_df.apply(lambda x: x['category'] in predictions[x['user']], axis=1).astype(int)

In [661]:
# Еще раз прогноз для пользователя №3
predictions[CUR_USER_ID]

[9,
 14,
 16,
 17,
 19,
 21,
 22,
 23,
 29,
 41,
 55,
 57,
 61,
 82,
 84,
 169,
 382,
 383,
 398,
 402,
 409,
 425,
 430]

In [662]:
# Проверяю глазами: если категория есть в предсказании, то 1. Иначе - 0
# 398 и 16 категории присутствуют - для них 1 в target - все верно.
sample_out_df[sample_out_df['user'] == CUR_USER_ID].head()

,id,target,user_category,user,category
56,3;134,0,"[3, 134]",3,134
57,3;398,1,"[3, 398]",3,398
58,3;399,0,"[3, 399]",3,399
59,3;15,0,"[3, 15]",3,15
60,3;16,1,"[3, 16]",3,16


In [663]:
sub_df = sample_out_df.copy()
sub_df = sub_df.loc[:, ['id', 'target']]
sub_df.to_csv('sub50.csv', index=False)   # Score = 0.45558
# Ура! Новый рекорд!

# Еще одна гипотеза

Теперь совместим результаты LTSM и популярные товары (по ним был хороший результат в первом ноутбуке)  
То есть возьмем результаты LTSM и для всех популярных товаров поставим 1

In [645]:
popular_df = df.groupby('cart')['order_completed_at'].count().to_frame().reset_index()  \
                .rename({'order_completed_at': 'orders_count'}, axis=1)  \
                .sort_values(by='orders_count', ascending=False)
popular_df = popular_df[popular_df['orders_count'] > 20000]

In [92]:
popular_df.head()

,cart,orders_count
57,57,108877
14,14,93957
61,61,91543
398,398,81694
23,23,71837


In [93]:
popular_categories = popular_df['cart'].values.tolist()

In [94]:
sub_df = sample_out_df.copy()

In [95]:
sub_df.loc[sub_df['category'].isin(popular_categories), 'target'] = 1

In [96]:
sub_df = sub_df.loc[:, ['id', 'target']]
sub_df.to_csv('sub19.csv', index=False)   # Score = 0.41062
# Улучшить не удалось

# Выводы:

1. Лучший результат был получен с помощью LSTM (Score = 0.45558)  
2. Хотя предсказывать просто самые популярные товары тоже дает неплохой результат (без всякого ML)  
3. Имеется очень большой потенциал для улучшения результата (хотя это потребует много времени):  
    - обучить LSTM на всех имеющихся данных (сейчас использовано только 5%)  
    - обучить LSTM хотя бы несколько эпох (сейчас была всего 1 эпоха)  
    - поэкспериментировать с порогом для отнесения к 1-му классу (порог 0.2 был взят на глазок)  
    - поэкспериментировать с learning rate, оптимизатором и функцией потерь  
    - поэкспериментировать с количеством нейронов в линейном слое модели (и с количеством этих слоев)  